### 转置卷积
> * 普通卷积不会增大输入的高宽，通常要么高宽不变，要么减半
> * 转置卷积则可以用来增大输入高宽
![](../images/Transposed-Convolution.png)
> * 从基本的转置卷积开始，设步幅为1且不设填充。假设有一个$n_h\times n_w$的输入张量和一个$k_h\times k_w$的卷积核。以步幅为1滑动卷积核窗口，每行$n_w$次，每列$n_h$次，共产生$n_h n_w$个中间结果。每个中间结果都是一个$(n_h+k_h-1)\times (n_w+k_w-1)$的张量，初始化为0。<font color='red'>为了计算每个中间张量，输入张量中的每个元素都要乘以卷积核，从而使得$k_h\times k_w$张量替换中间张量的一部分。请注意，每个张量被替换部分的位置与输入张量中元素的位置相对应。</font>最后，所有中间结果相加以获得最终结果。
> * 输出为$Y[i:i+h, j:j+w] += X[i,j]\cdot K$

#### 为什么称之为转置
> * 对于卷积$Y=X\bigstar W$
>> * 可以对$W$构造一个V，使得卷积等价于矩阵乘法$Y^\prime=VX^\prime$
>> * 这里$Y^\prime,X^\prime$是Y,X对应的向量版本
> * 转置卷积则等价于$Y^\prime=V^TX^\prime$
> * 如果卷积将输入从$(h,w)$变成了$(h^\prime,w^\prime)$
>> * 同样超参数的转置卷积则从$(h^\prime,w^\prime)$变成了$(h,w)$

In [2]:
import torch
from torch import nn
from d2l import limutorch as d2l

In [3]:
"""实现基本的转置卷积运算"""
def trans_conv(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i:i+h, j:j+w] += X[i, j] * K
    return Y

In [5]:
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

In [6]:
"""使用高级API获得相同的结果"""
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

#### 转置卷积的填充
与常规卷积不同，在转置卷积中，填充被应用于输出中（常规卷积将填充应用于输出中）。例如，<font color='red'>当将宽和高两侧的填充数指定为1时，转置卷积的输出中将删除第一和最后的行列</font>。加了填充会将输出变小。

In [7]:
"""填充，步幅和多通道"""
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)

#### 转置卷积的步幅
在转置卷积中，步幅被指定为中间结果（输出），而不是输入。下图是将步幅从1变为2增加的中间变量的高和权重。
![](../images/Stride-of-Transposed-Convolution.png)

In [8]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)

In [9]:
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape

True

In [10]:
"""与矩阵变换的联系"""
X = torch.arange(9.0).reshape(3, 3)
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
Y = d2l.corr2d(X, K)
Y

tensor([[27., 37.],
        [57., 67.]])

In [11]:
def kernel2matrix(K):
    k, W = torch.zeros(5), torch.zeros((4, 9))
    k[:2], k[3:5] = K[0, :], K[1, :]
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W
W = kernel2matrix(K)
W

tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

In [12]:
Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[True, True],
        [True, True]])

In [15]:
Z = trans_conv(Y, K)
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3), Z

(tensor([[True, True, True],
         [True, True, True],
         [True, True, True]]),
 tensor([[ 27.,  91.,  74.],
         [138., 400., 282.],
         [171., 429., 268.]]))

#### 重新排列输入和核
> * 当填充为0步幅为1时
>> * 将输入填充k-1（k是核窗口）
>> * 将核矩阵上下，左右翻转
>> * 然后做正常卷积（填充0， 步幅1）
![](../images/Rearrange1.png)
> * 当填充为p，步幅为1时
>> * 将输入填充$k-p-1$（k是核窗口）
>> * 将和举证上下，左右翻转
>> * 然后做正常卷积（填充0，步幅1）
![](../images/Rearrange2.png)
> * 当填充为p步幅为s时
>> * 在行和列之间插入$s-1$行或列
>> * 将输入填充$k-p-1$（k是核窗口）
>> * 将核矩阵上下，左右翻转
>> * 然后正常卷积（填充0，步幅1）
![](../images/Rearrange3.png)

#### 形状换算
> * 输入高（宽）为n，核k，填充p，步幅s
> * 转置卷积形状，高宽皆为：$n^\prime = sn+k-p-s$
>> * 卷积形状，高宽皆为：$n^\prime=\llcorner(n-k+p+s)/s\lrcorner$$\rightarrow n\geqslant sn^\prime+k-p-s$,即原始图像形状大小大于等于转置卷积后的形状大小
> * 如果让高宽成倍增加，那么$k=p+s$
> * <font color='red'>在实际运算过程，p的大小为设置值的两倍，因为p=k，意味着上下左右各加k行，总计添加2k行</font>

#### 转置卷积和反卷积的关系
> * 数学上的反卷积是指卷积的逆运算
>> * 如果Y=conv(X,K),那么X=deconv(Y,K)m
> * 反卷积很少用在深度学习中
>> * 反卷积神经网络指用了转置卷积的神经网络
> * <font color='red'>转置卷积只是获得卷积前相同的形状，而反卷积在获得相同形状的同时，也获得相同的数据</font>

### 总结
> * 转置卷积是一种变化了输入和核的卷积，来达到上采样的目的
> * 转置卷积不等同于数学上的反卷积操作